ในเคสนี้เราจะเน้นการใช้ Tensorboard ดูการทำงานภายในโมเดล โดยใช้ Dataset MNIST ส่วนรายละเอียดการเทรนโมเดล Image Classifier ชุดข้อมูล MNIST ขอให้ดู ep ก่อน ๆ

# 0. Magic Commands


เพิ่ม magic command ของ tensorboard

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

%load_ext tensorboard

# 1. Import Library

เพิ่ม import tensorboard callbacks

In [2]:
from fastai import *
from fastai.vision import *
from fastai.metrics import accuracy
import time

from fastai.callbacks.tensorboard import *

# 2. ข้อมูล

ใช้ชุดข้อมูล Dataset [MNIST](http://yann.lecun.com/exdb/mnist/) ตัวเลขอารบิคเขียนด้วยลายมือ

In [3]:
path = untar_data(URLs.MNIST)

# 3. เตรียมข้อมูล



In [4]:
batchsize = 64
np.random.seed(0)
transform = get_transforms(do_flip=False)

databunch = ImageDataBunch.from_folder(path, train='training', 
                                       valid_pct=0.2, size=28, 
                                       ds_tfms=transform, bs=batchsize, 
                                       num_workers=8).normalize()

# 4. สร้างโมเดล

In [5]:
learner = cnn_learner(databunch, models.resnet18, metrics=accuracy)

สร้าง Callback เพื่อส่งข้อมูลภายในโมเดล ไปให้ Tensorboard

In [6]:
proj_id = '07a-tensorboard-mnist'
tboard_path = Path('data/tensorboard/' + proj_id)
learner.callback_fns.append(partial(LearnerTensorboardWriter, 
                                    base_dir=tboard_path, 
                                    name=f'Learner{time.time()}'))
print(f"tensorboard --logdir={tboard_path}")

tensorboard --logdir=data/tensorboard/07a-tensorboard-mnist


สำหรับ clear logs และ kill process tensorboard

In [7]:
# ! rm -rf $tboard_path

เราสามารถดูเปิด Tensorboard ได้ 2 ทาง

1. ใน Notebook ให้รัน cell magic command ด้านล่าง

2. นอก Notebook เปิด Shell ใหม่ แล้วรันคำสั่ง tensorboard --logdir=./data/tensorboard/07a-tensorboard-mnist แล้วเปิดลิงค์ http://localhost:6006

*เราอาจจะต้องรอให้เทรนผ่าน Epoch แรกไปก่อน ถึงจะเห็นกราฟ*

In [ ]:
%tensorboard --logdir=$tboard_path

![title](https://www.bualabs.com/wp-content/uploads/2019/08/bualabs-tensorboard-screenshot07-768x494.jpg)

# 5. เริ่มต้นเทรนโมเดล

เทรนด้วยค่า Default ไป 4 Cycle

In [9]:
learner.fit_one_cycle(4)

epoch,train_loss,valid_loss,accuracy,time
0,0.683487,0.503954,0.839214,04:07
1,0.323790,0.197833,0.936571,04:41
2,0.236515,0.135550,0.957643,03:31


KeyboardInterrupt: 

# 6. เทรนต่อ

In [ ]:
learner.fit_one_cycle(3, max_lr=slice(1e-6, 3e-3))

## สำเร็จแล้ว ##

เราสามารถใช้ Tensorboard ดูสถิติภายในโมเดล เพื่อใช้ในการ Debug และ Hyperparameter Tuning ไปถึงนำไป Present ได้อย่างสวยงาม


# 7. สรุป


1. เราสามารถเรียกใช้ Tensorboard แสดงกราฟสวยงาม ได้ด้วยโค้ด callback ไม่กี่บรรทัด
2. แต่การตีความกราฟที่เห็นการทำงานภายในโมเดล ต้องมีความเข้าใจ การออกแบบ โมเดล Layer แบบต่าง ๆ ที่เราจะอธิบายต่อไป

# Credit 

* https://github.com/fastai/fastai/blob/master/fastai/callbacks/tensorboard.py
* https://www.tensorflow.org/tensorboard/r2/tensorboard_in_notebooks
